# 1: Decision Trees
Decision trees are a powerful and commonly used machine learning technique.

If we wanted to optimize our chances of surviving a bear encounter, we could construct a decision tree to tell us what action to take.As our dataset gets larger though, this gets less and less practical. What if we had 10000 rows and 10 variables? Would you want to look through all the possibilities to construct a tree?

This is where the decision tree machine learning algorithm can help. It enables us to automatically construct a decision tree to tell us what outcomes we should predict in certain situations.

The decision tree algorithm is a supervised learning algorithm -- we first construct the tree with historical data, and then use it to predict an outcome. One of the major advantages of decision trees is that they can pick up nonlinear interactions between variables in the data that linear regression can't.

Trees can be used for classification or regression problems.

In [3]:
import pandas as pd
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
           'marital_status', 'occupation', 'relationship', 'race', 'sex',
           'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'high_income']
income = pd.read_csv('adult.data',names=columns)

In [5]:
print income.head()

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country high_income  
0          2174             0              40   United-States   

# 3: Converting categorical variables
we need to convert the categorical variables in our dataset to numeric variables. This involves assigning a number to each category label, and converting all the labels in a column to numbers.

We can use the Categorical.from_array method from Pandas to perform the conversion to numbers.

In [8]:
col = pd.Categorical.from_array(income['workclass'])
income['workclass'] = col.codes
print(income['workclass'].head())

target_col = ['education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country', 'high_income']

for target in target_col:
    col = pd.Categorical.from_array(income[target])
    income[target] = col.codes

0    7
1    6
2    4
3    4
4    4
Name: workclass, dtype: int8


# 4: Split points
A decision tree is constructed through a series of nodes and branches. A node is where we split the data based on a variable, and a branch is one side of the split. Here's an example:図

This is exactly how a decision tree works -- we keep splitting the data based on variables. As we do this, the tree gets deeper and deeper. The tree we made above is two levels deep, because it has one split, and two "levels" of nodes. 

# 5: Performing a split
In the diagram above, we can split the dataset into two portions based on whether the individual works in the private sector or not.

In [12]:
private_incomes = income[income['workclass'] == 4]
public_incomes = income[income['workclass'] != 4]

# 6: Thinking about the data
When we make a split, some rows will go to the right, and some rows will go to the left. As we build the tree deeper and deeper, each node will "receive" less and less rows.

# 7: Rationale behind splitting
The nodes at the bottom of the tree, when we decide to stop splitting, are called terminal nodes, or leaves.Our goal is to ensure that we can make a prediction on future data. In order to do this, all rows in each leaf must have only one value for our target column.

In order to be able to make this prediction, all leaves need to have rows with only a single value for high_income.

# 8: Entropy
we need a metric for how "together" the different values in the high_income column are.
Entropy, which is not to be confused with entropy from physics, comes from information theory.Entropy can also be thought of in terms of information. If we flip a coin where both sides are heads, we know upfront that the result will be heads. There's much more complexity, especially when you get to cases with more than two possible outcomes, or differential probabilities.

$$-\sum_{i=1}^{c} {\mathrm{P}(x_i) \log_b \mathrm{P}(x_i)}$$

We iterate through each unique value in a single column (in this case, high_income), and assign it to i.